## Let's plot some of the columns

Linear Regression From Scratch

In [1]:
from __future__ import annotations

import math
import time

from pandas import DataFrame
import pandas as pd
import random
from sklearn.metrics import accuracy_score
import time
import numpy as np
import pandas as pd
from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [2]:
# !wget https: // archive.ics.uci.edu / ml / machine-learning-databases / 00332 / OnlineNewsPopularity.zip
# !unzip OnlineNewsPopularity.zip

Format properly the names of the columns and remove the first two columns

In [3]:
#VARIABLES
BINARY_LABEL = True
NORMALIZE = True

In [4]:
df = pd.read_csv('OnlineNewsPopularity/OnlineNewsPopularity.csv')
df = df.rename(columns=lambda x: x.strip())
df = df.iloc[:, 2:]

## Let's plot some of the columns

In [5]:
import matplotlib.pyplot as plt

columns_to_plot = [
    'n_tokens_title',
    'num_videos',
    'num_imgs',
    'num_keywords',
    'data_channel_is_world',
    'rate_negative_words',
    'self_reference_avg_sharess',
]
#
# fig, ax = plt.subplots(len(columns_to_plot), 1, figsize=(20, 20))
#
# for i, column in enumerate(columns_to_plot, 0):
#     ax[i].hist(df[column])
#     ax[i].title.set_text(column)

# plt.show()

In [6]:
#calculate median for each column
median = df.median(axis=0)
avg = df.mean(axis=0)
print(avg)
print(median)
# compute the median of each attribute
medians = df.median()

# discretize each attribute to 0 or 1 based on the median
# for column in df.columns:
#     df[column] = (df[column] >= medians[column]).astype(int)

n_tokens_title                       10.398749
n_tokens_content                    546.514731
n_unique_tokens                       0.548216
n_non_stop_words                      0.996469
n_non_stop_unique_tokens              0.689175
num_hrefs                            10.883690
num_self_hrefs                        3.293638
num_imgs                              4.544143
num_videos                            1.249874
average_token_length                  4.548239
num_keywords                          7.223767
data_channel_is_lifestyle             0.052946
data_channel_is_entertainment         0.178009
data_channel_is_bus                   0.157855
data_channel_is_socmed                0.058597
data_channel_is_tech                  0.185299
data_channel_is_world                 0.212567
kw_min_min                           26.106801
kw_max_min                         1153.951682
kw_avg_min                          312.366967
kw_min_max                        13612.354102
kw_max_max   

In [7]:
data = np.array(df)
if NORMALIZE:
    # normalize the data
    data = (data - data.mean(axis=0)) / data.std(axis=0)
x = data[:, :-1]
# converting the last column to boolean
if BINARY_LABEL:
    if not NORMALIZE:
        y = np.array([elem >= 1400 for elem in data[:, -1]])
    else:
        y = np.array([e >= 0 for e in data[:, -1]])  #TODO check
else:
    y = np.array(data[:, -1])
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.2, random_state=1)

In [12]:
from sklearn.metrics import log_loss


def analyze_pred(pred, truth):
    # Print the mean squared error and R-squared score
    print('Binary cross entropy:', log_loss(truth, pred))
    print(np.mean(pred))
    print(np.mean(truth))
    # bin_pred = pred >= 0
    # bin_truth = truth >= 0
    print('Accuracy:', accuracy_score(truth, pred))

In [10]:
def test_model(model, train_x, train_y, test_x, test_y):
    time_start = time.time()
    model.fit(train_x, train_y)
    print("Time taken to train the model: ", time.time() - time_start)
    pred = model.predict(test_x)
    analyze_pred(pred, test_y)

In [13]:
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

regressor = LogisticRegression(max_iter=1000)

# Print the regression coefficients and intercept
# print('Coefficients:', regressor.coef_)
# print('Intercept:', regressor.intercept_)
test_model(regressor, train_x, train_y, test_x, test_y)


Time taken to train the model:  1.2596004009246826
Binary cross entropy: 7.282372648425485
0.023332072140244672
0.20267372934796318
Accuracy: 0.7979568671963677


Linear Regression From Scratch

In [18]:
class ScratchLogisticRegression:

    def __init__(self, lr=0.01, n_iters=1000, regularization=None):
        self.lr = lr
        self.n_iters = n_iters
        self.weights = None
        self.bias = None
        self.regularization = regularization
        self.regularization_param = 0.1
        self.alpha = 0.1

    def _sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def fit(self, X, y):
        n_samples, n_features = X.shape

        self.weights = np.zeros(n_features)
        self.bias = 0

        for _ in range(self.n_iters):
            y_pred = np.dot(X, self.weights) + self.bias
            y_pred = self._sigmoid(y_pred)

            if self.regularization == 'l1':
                dw = (1 / n_samples) * (np.dot(X.T, (y_pred - y)) + np.sign(self.weights) * self.regularization_param)
                db = (1 / n_samples) * (np.sum(y_pred - y) + np.sign(self.bias) * self.regularization_param)

            elif self.regularization == 'l2':
                dw = (1 / n_samples) * (np.dot(X.T, (y_pred - y)) + 2 * self.weights * self.regularization_param)
                db = (1 / n_samples) * (np.sum(y_pred - y) + 2 * self.bias * self.regularization_param)
            # elif self.regularization == 'elasticnet':
            #     dw = (1 / n_samples) * (np.dot(X.T, (y_pred - y)) + 2  * self.weights*self.regularization_param + np.sign(self.weights) * self.regularization_param)
            #     db = (1 / n_samples) * (np.sum(y_pred - y) + 2 * self.bias*self.regularization_param + np.sign(self.bias) * self.regularization_param)
            else:
                dw = (1 / n_samples) * np.dot(X.T, (y_pred - y))
                db = (1 / n_samples) * np.sum(y_pred - y)

            self.weights -= self.lr * dw
            self.bias -= self.lr * db

    def predict(self, X):
        y_pred = np.dot(X, self.weights) + self.bias
        y_pred = self._sigmoid(y_pred)
        return y_pred>=0.5

In [23]:
regressor = ScratchLogisticRegression(regularization='l1')

# Print the regression coefficients and intercept
# print('Coefficients:', regressor.coef_)
# print('Intercept:', regressor.intercept_)
test_model(regressor, train_x, train_y, test_x, test_y)


Time taken to train the model:  2.9948294162750244
Binary cross entropy: 7.341468056933308
0.01412536259301299
0.20267372934796318
Accuracy: 0.7963173161811073
